In [4]:
from HPCv3.Gym.utils import learn
from HPCv3.Utils import get_logger
from HPCv3.Simulator.Simulator import Simulator, run_simulation
from HPCv3.Gym.gym import HPCGymEnv
from datetime import datetime
from agent.agent import Agent
from agent.critic import Critic
from agent.memory import PPOMemory
import torch as T

logger = get_logger("runner", level="INFO", log_file="results/simulation.log")

In [5]:
# === Configuration ===
workload_path = "workloads/generated.json"  # Path to platform JSON file
platform_path = "platforms/generated.json"  # Path to output folder
output_path = "results/generated"  # Scheduling algorithm name
algorithm = "fcfs"  # Simulation timeout in seconds (optional)
# Time in format YYYY-MM-DD HH:MM:SS (optional). If not provided, starts at 0.
timeout = None
start_time = None
RL = True  # Reinforcement Learning mode

In [ ]:
# === Initialize simulator ===
if start_time is not None:
    start_time = int(datetime.strptime(
        start_time, "%Y-%m-%d %H:%M:%S").timestamp())
    human_readable = True
else:
    start_time = 0
    human_readable = False


# === Main ===
if RL:
    # === RL parameters ===
    learning_rate = 0.0003
    n_heads = 4
    n_gae_layers = 2
    input_dim = 11
    embed_dim = 16
    gae_ff_hidden = 256
    tanh_clip = 10.0
    mini_batch_size = 64
    device = "cuda"

    agent = Agent(n_heads, n_gae_layers, input_dim,
                  embed_dim, gae_ff_hidden, tanh_clip, device)
    agent_opt = T.optim.Adam(agent.parameters(), lr=learning_rate)

    critic = Critic(n_heads, n_gae_layers, input_dim,
                    embed_dim, gae_ff_hidden, tanh_clip, device)
    critic_opt = T.optim.Adam(critic.parameters(), lr=learning_rate)

    memory = PPOMemory(mini_batch_size)

    simulator = Simulator(workload_path, platform_path,
                          start_time, algorithm, rl=True, agent=agent)

    env = HPCGymEnv(simulator)
    simulator.start_simulator()
    while simulator.is_running:
        # Steps 1-3: agent selects action, environment executes it
        next_features, reward, done, _memory = env.step(simulator.agent)
        # Step 4: Critic evaluates state value
        value_current = critic(_memory['features'])
        value_next = critic(next_features)

        # Step 5: Compute Advantage
        gamma = 0.99
        advantage = reward + gamma * value_next - value_current

        # Step 6: Store experience (optional)
        memory.store_memory(
            features=_memory['features'],
            mask=_memory['mask'],
            action=_memory['actions'],
            reward=_memory['reward'],
        )

        # Step 7: Update Actor & Critic immediately
        # CHANGE THE LEARNING
        saved_experiences = memory.get_memory()
        learn(simulator.agent, agent_opt, critic,
              critic_opt, not simulator.is_running, saved_experiences, next_features)
        # # or agent.get_log_prob(obs_tensor, action)
        # curr_log_prob = action_probs
        # actor_loss = -(curr_log_prob * advantage.detach()).mean()
        # critic_loss = (advantage ** 2).mean()

        # # Gradient step
        # agent_opt.zero_grad()
        # actor_loss.backward()
        # agent_opt.step()

        # critic_opt.zero_grad()
        # critic_loss.backward()
        # critic_opt.step()

    # Step 8: No need to clear memory each step if you want to keep history
    memory.clear_memory()  # optional


else:
    simulator = Simulator(workload_path, platform_path, start_time, algorithm)
    run_simulation(simulator, output_path)

2025-09-05 17:42:57 | INFO | runner:98 - [Time=0.00] type=simulation_start
2025-09-05 17:42:57 | INFO | runner:98 - [Time=11.70] job_id=1 type=arrival res=4 subtime=11.7 walltime=49 profile=100 user_id=0
2025-09-05 17:42:57 | INFO | runner:98 - [Time=11.70] type=CALL_RL
2025-09-05 17:42:57 | INFO | runner:109 - Action taken: tensor([[[0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160]]], device='cuda:0',
       grad_fn=<DifferentiableGraphBackward>)
2025-09-05 17:42:57 | INFO | runner:111 - Switch off: []
2025-09-05 17:42:57 | INFO | runner:112 - Switch on: []
2025-09-05 17:42:57 | INFO | runner:98 - [Time=11.70] type=swit

torch.Size([1])
torch.Size([1, 1, 16, 2])


2025-09-05 17:42:57 | INFO | runner:98 - [Time=16.49] job_id=3 type=arrival res=2 subtime=16.49 walltime=49 profile=100 user_id=0
2025-09-05 17:42:57 | INFO | runner:98 - [Time=16.49] type=CALL_RL
2025-09-05 17:42:57 | INFO | runner:109 - Action taken: tensor([[[0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160]]], device='cuda:0',
       grad_fn=<DifferentiableGraphBackward>)
2025-09-05 17:42:57 | INFO | runner:111 - Switch off: []
2025-09-05 17:42:57 | INFO | runner:112 - Switch on: []
2025-09-05 17:42:57 | INFO | runner:98 - [Time=16.49] type=switch_off nodes=[]
2025-09-05 17:42:57 | INFO | runner:98 - [Time=16.49] type

torch.Size([2])
torch.Size([2, 1, 16, 2])


2025-09-05 17:42:58 | INFO | runner:98 - [Time=27.53] job_id=5 type=arrival res=3 subtime=27.53 walltime=50 profile=100 user_id=0
2025-09-05 17:42:58 | INFO | runner:98 - [Time=27.53] type=CALL_RL
2025-09-05 17:42:58 | INFO | runner:109 - Action taken: tensor([[[0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160]]], device='cuda:0',
       grad_fn=<DifferentiableGraphBackward>)
2025-09-05 17:42:58 | INFO | runner:111 - Switch off: []
2025-09-05 17:42:58 | INFO | runner:112 - Switch on: []
2025-09-05 17:42:58 | INFO | runner:98 - [Time=27.53] type=switch_off nodes=[]
2025-09-05 17:42:58 | INFO | runner:98 - [Time=27.53] type

...

2025-09-05 17:43:01 | INFO | runner:98 - [Time=147.00] job_id=37 type=arrival res=3 subtime=147.0 walltime=49 profile=100 user_id=0
2025-09-05 17:43:01 | INFO | runner:98 - [Time=147.00] type=CALL_RL
2025-09-05 17:43:01 | INFO | runner:109 - Action taken: tensor([[[0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160]]], device='cuda:0',
       grad_fn=<DifferentiableGraphBackward>)
2025-09-05 17:43:01 | INFO | runner:111 - Switch off: []
2025-09-05 17:43:01 | INFO | runner:112 - Switch on: []
2025-09-05 17:43:01 | INFO | runner:98 - [Time=147.00] type=switch_off nodes=[]
2025-09-05 17:43:01 | INFO | runner:98 - [Time=147.00]

torch.Size([18])
torch.Size([18, 1, 16, 2])


2025-09-05 17:43:02 | INFO | runner:98 - [Time=165.65] job_id=39 type=arrival res=5 subtime=165.65 walltime=52 profile=100 user_id=0
2025-09-05 17:43:02 | INFO | runner:98 - [Time=165.65] type=CALL_RL
2025-09-05 17:43:02 | INFO | runner:109 - Action taken: tensor([[[0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160],
         [0.4840, 0.5160]]], device='cuda:0',
       grad_fn=<DifferentiableGraphBackward>)
2025-09-05 17:43:02 | INFO | runner:111 - Switch off: []
2025-09-05 17:43:02 | INFO | runner:112 - Switch on: []
2025-09-05 17:43:02 | INFO | runner:98 - [Time=165.65] type=switch_off nodes=[]


torch.Size([19])
torch.Size([19, 1, 16, 2])


2025-09-05 17:43:02 | INFO | runner:98 - [Time=165.65] type=switch_on nodes=[]
2025-09-05 17:43:02 | INFO | runner:98 - [Time=170.20] job_id=40 type=arrival res=3 subtime=170.2 walltime=49 profile=100 user_id=0
2025-09-05 17:43:02 | INFO | runner:98 - [Time=170.20] type=CALL_RL
